In [1]:
import json
import shutil
import traceback
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import src.api.controllers.generate_embeddings as generate_embeddings
import torch
import torchvision.transforms.functional as F
from sqlalchemy.orm import Session
from src.config import GAZE_FOVEA_FOV, TOBII_FOV_X
from src.db import engine
from src.db.models import Recording, SimRoomClass
from src.api.controllers.gaze_segmentation import (
    get_gaze_points,
    match_frames_to_gaze,
    parse_gazedata_file,
    mask_was_viewed
)
from src.utils import cv2_video_fps, cv2_video_frame_count, cv2_video_resolution
from torchvision.ops import masks_to_boxes
from torchvision.transforms import InterpolationMode
from ultralytics import FastSAM
from tqdm import tqdm

2025-04-06 23:00:58.684474: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 23:00:58.697273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743973258.707583  291001 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743973258.710848  291001 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 23:00:58.725218: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
with open("experiment_metadata.json") as file:
    experiment_metadata = json.load(file)
    trial_recordings_metadata = experiment_metadata["trial_recordings_metadata"]
    trial_recording_uuids = list(trial_recordings_metadata.keys())
    labeling_same_background_uuid = experiment_metadata["labeling_same_background_uuid"]
    labeling_diff_background_uuid = experiment_metadata["labeling_diff_background_uuid"]

with Session(engine) as session:
    trial_recordings = (
        session.query(Recording).filter(Recording.uuid.in_(trial_recording_uuids)).all()
    )

In [3]:
dinov2 = generate_embeddings.load_model()

In [ ]:
GAZE_SEGMENTATION_RESULTS_PATH = Path("data/gaze_segmentation_results")

def process_recording(recording: Recording):
    recording_uuid = recording.uuid
    gaze_segmentation_results_path = GAZE_SEGMENTATION_RESULTS_PATH / recording_uuid
    
    gaze_segmentation_results = list(gaze_segmentation_results_path.iterdir())
    gaze_segmentation_results.sort(key=lambda x: int(x.stem))
    gaze_segmentation_results = [
        np.load(result, allow_pickle=True) for result in gaze_segmentation_results
    ]

    grounding_stats = pd.DataFrame(
        columns=[
            "frame_idx",
            "object_id",
            "class_id",
            "avg_distance",
            "min_distance",
            "max_distance",
            "var_distance",
        ]
    )

    result_rows = []
    for result in gaze_segmentation_results:
        frame_idx = result["frame_idx"]
        rois = result["rois"]
        object_ids = result["object_ids"]

        # Get embeddings (assuming one batch is returned)
        embeddings, _, _ = list(generate_embeddings.get_embeddings(dinov2, rois))[0]
        per_roi_distances, per_roi_class_ids = generate_embeddings.search_index(
            index, embeddings, k=50
        )

        for i, roi in enumerate(rois):  # iterate over each ROI
            object_id = object_ids[i]
            distances = per_roi_distances[i]
            class_ids = per_roi_class_ids[i]

            # Group distances by class using defaultdict for conciseness
            class_to_dists = defaultdict(list)
            for cid, d in zip(class_ids, distances, strict=False):
                class_to_dists[cid].append(d)

            # For each class, compute statistics and add a row
            for cid, dists in class_to_dists.items():
                rows.append({
                    "frame_idx": frame_idx,
                    "object_id": object_id,
                    "class_id": cid,
                    "avg_distance": np.mean(dists),
                    "min_distance": np.min(dists),
                    "max_distance": np.max(dists),
                    "var_distance": np.var(dists),
                })

    grounding_stats = pd.DataFrame(rows)
        
